In [1]:
# import keras.callbacks
from music21 import *

import re
import xml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dense, Dropout
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [62]:
in_out_neurons = 5
hidden_neurons = 300

model = Sequential()  
model.add(LSTM(hidden_neurons, return_sequences=False, input_shape=(None, in_out_neurons)))
model.add(Dense(in_out_neurons, input_dim=hidden_neurons))  
model.add(Activation("linear"))  
model.compile(loss="mean_squared_error", optimizer="rmsprop")  

In [2]:
with open('environment.txt') as f:
    env = f.readlines()
# you may also want to remove whitespace characters like \n at the end of each line
env = [x.strip() for x in env]

us = environment.UserSettings()
us['musicxmlPath'] = env[0]
us['musescoreDirectPNGPath'] = env[0]



In [3]:
# fix random seed for reproducibility
np.random.seed(7)

In [4]:
bachRegex = re.compile(r'.*bwv.*', re.IGNORECASE)
bd = corpus.search(bachRegex)

In [5]:
def convertSongToDataFrame(song):
    df = pd.DataFrame()
    
    currentKeySignature = None
    currentTimeSignature = None
    for measure in song.measures(0, None, collect=('TimeSignature','KeySignature')):
        if measure.keySignature:
            currentKeySignature = measure.keySignature
        if measure.timeSignature:
            currentTimeSignature = measure.timeSignature.ratioString
        for i in measure.iter:
            if type(i) is chord.Chord:
                df2 = pd.DataFrame([currentKeySignature, currentTimeSignature, roman.romanNumeralFromChord(i, currentKeySignature).figure, i.duration.quarterLength, i.offset + measure.offset])
                df = df.append(df2[0], ignore_index = True)
    return df

In [8]:
# only run if you want to create csv files
import os.path
import os
if not os.path.exists('./data'):
    os.makedirs("./data")
for s in bd:
    fname = "./data/" + s.sourcePath.split("/")[1]+'.csv'
    if os.path.isfile(fname):
        continue
    df = convertSongToDataFrame(s.parse().chordify())
    df.to_csv(fname, sep=',')
    
#failed at bwv276.mxl

AttributeError: 'KeySignature' object has no attribute 'tonic'

In [34]:
from functools import reduce
scores = []
import glob, os
os.chdir("C:/Users/errpo/Desktop/projects/project folder/music_prediction/data")
for file in glob.glob("*.csv"):
    scores.append(file)
scoreDf = []

def addColumnNames(df):
    df.columns = ['keySignature', 'timeSignature', 'chord', 'duration', 'measure']
    #return df

for i in scores:
    score = pd.read_csv(i,usecols=[1,2,3,4,5])
    addColumnNames(score)
    scoreDf.append(score)
    
def createCategories(df):
    addColumnNames(df)
    keySignatures = pd.DataFrame([df['keySignature'].astype('category').cat.codes.unique()], columns=df['keySignature'].unique()).to_dict(orient = 'records')
    timeSignatures = pd.DataFrame([df['timeSignature'].astype('category').cat.codes.unique()], columns=df['timeSignature'].unique()).to_dict(orient = 'records')
    chords = pd.DataFrame([df['chord'].astype('category').cat.codes.unique()], columns=df['chord'].unique()).to_dict(orient = 'records')
    df['keySignature'] = df['keySignature'].astype('category').cat.codes
    df['timeSignature'] = df['timeSignature'].astype('category').cat.codes
    df['chord'] = df['chord'].astype('category').cat.codes
    return keySignatures, timeSignatures, chords

allScores = pd.concat(scoreDf)
(keySignatures, timeSignatures, chords) = createCategories(allScores)

# keySignatures = pd.DataFrame([allScores['keySignature'].astype('category').cat.codes.unique()], columns=allScores['keySignature'].unique()).to_dict(orient = 'records')
# timeSignatures = pd.DataFrame([allScores['timeSignature'].astype('category').cat.codes.unique()], columns=allScores['timeSignature'].unique()).to_dict(orient = 'records')
# chords = pd.DataFrame([allScores['chord'].astype('category').cat.codes.unique()], columns=allScores['chord'].unique()).to_dict(orient = 'records')
# allScores['keySignature'] = allScores['keySignature'].astype('category').cat.codes
# allScores['timeSignature'] = allScores['timeSignature'].astype('category').cat.codes
# allScores['chord'] = allScores['chord'].astype('category').cat.codes
# addColumnNames(allScores)

# print(scoreDf[1])
scores = pd.concat(scoreDf[0:10])
# scores = scoreDf[0:10]
testScores = pd.concat(scoreDf[11:12])

avgScoreLength = math.floor(reduce((lambda x,y: x+ len(y)), scoreDf, 0)/len(scoreDf))
#replaceKeySignatures(scores)
keySignatures[0]
def categorize(df):
    df['keySignature']=df['keySignature'].replace(keySignatures[0])
    df['timeSignature']=df['timeSignature'].replace(timeSignatures[0])
    df['chord']=df['chord'].replace(chords[0])

categorize(scores)
categorize(testScores)

In [102]:
# Load O Haupt voll Blut und Wunden (chorales)

part1 = convertSongToDataFrame(corpus.parse("bwv244.15.mxl").chordify())#15, 17, 54, 62
part2 = convertSongToDataFrame(corpus.parse("bwv244.17.mxl").chordify())
part3 = convertSongToDataFrame(corpus.parse("bwv244.54.mxl").chordify())
part4 = convertSongToDataFrame(corpus.parse("bwv244.62.mxl").chordify())
part5 = convertSongToDataFrame(corpus.parse("bwv244.3.mxl").chordify())
part6 = convertSongToDataFrame(corpus.parse("bwv244.10.mxl").chordify())
part7 = convertSongToDataFrame(corpus.parse("bwv244.25.mxl").chordify())
part8 = convertSongToDataFrame(corpus.parse("bwv244.32.mxl").chordify())
part9 = convertSongToDataFrame(corpus.parse("bwv244.37.mxl").chordify())
part10 = convertSongToDataFrame(corpus.parse("bwv244.40.mxl").chordify())

# Resequence measure numbers

endMeasure = math.ceil(part1.iloc[-1,4] + part1.iloc[-1,3])
part2[4] = part2[4] + endMeasure
endMeasure = math.ceil(part2.iloc[-1,4] + part2.iloc[-1,3])
part3[4] = part3[4] + endMeasure
endMeasure = math.ceil(part3.iloc[-1,4] + part3.iloc[-1,3])
part4[4] = part4[4] + endMeasure
endMeasure = math.ceil(part4.iloc[-1,4] + part4.iloc[-1,3])
part5[4] = part5[4] + endMeasure
endMeasure = math.ceil(part5.iloc[-1,4] + part5.iloc[-1,3])
part6[4] = part6[4] + endMeasure
endMeasure = math.ceil(part6.iloc[-1,4] + part6.iloc[-1,3])
part7[4] = part7[4] + endMeasure
endMeasure = math.ceil(part7.iloc[-1,4] + part7.iloc[-1,3])
part8[4] = part8[4] + endMeasure
endMeasure = math.ceil(part8.iloc[-1,4] + part8.iloc[-1,3])
part9[4] = part9[4] + endMeasure
endMeasure = math.ceil(part9.iloc[-1,4] + part9.iloc[-1,3])
part10[4] = part10[4] + endMeasure

In [103]:
allParts = []
allParts.append(part1)
allParts.append(part2)
allParts.append(part3)
allParts.append(part4)
allParts.append(part5)
allParts.append(part6)
allParts.append(part7)
allParts.append(part8)
allParts.append(part9)
allParts.append(part10)
allParts = pd.concat(allParts)
allParts = allParts.reset_index(drop=True)

allParts[0] = pd.Categorical(allParts[0], allParts[0].unique())
allParts[1] = pd.Categorical(allParts[1], allParts[1].unique())
allParts[2] = pd.Categorical(allParts[2], allParts[2].unique())


allParts[0] = allParts[0].cat.rename_categories(range(0, allParts[0].unique().size))
allParts[1] = allParts[1].cat.rename_categories(range(0, allParts[1].unique().size))
allParts[2] = allParts[2].cat.rename_categories(range(0, allParts[2].unique().size))
allParts

#createCategories(allParts)

,0,1,2,3,4
0,0,0,0,1.0,0.0
1,0,0,1,1.0,1.0
2,0,0,2,1.0,2.0
3,0,0,3,0.5,3.0
4,0,0,4,0.5,3.5
5,0,0,5,0.5,4.0
6,0,0,0,0.5,4.5
7,0,0,6,1.0,5.0
8,0,0,7,1.0,6.0
9,0,0,0,1.0,7.0


In [66]:
# copied from http://danielhnyk.cz/predicting-sequences-vectors-keras-using-rnn-lstm/
def _load_data(data, n_prev = 100):  
    """
    data should be pd.DataFrame()
    """

    docX, docY = [], []
    for i in range(len(data)-n_prev):
        docX.append(data.iloc[i:i+n_prev].as_matrix())
        docY.append(data.iloc[i+n_prev].as_matrix())
    alsX = np.array(docX)
    alsY = np.array(docY)

    return alsX, alsY

def train_test_split(df, test_size=0.1, slice_size=100):  
    """
    This just splits data to training and testing parts
    """
    ntrn = round(len(df) * (1 - test_size))

    X_train, y_train = _load_data(df.iloc[0:ntrn], n_prev = slice_size)
    X_test, y_test = _load_data(df.iloc[ntrn:], n_prev = slice_size)

    return (X_train, y_train), (X_test, y_test)

In [104]:
#(X_train, y_train), (X_test, y_test) = train_test_split(scores) 
# (X_train, y_train) = _load_data(scores[0:], n_prev=50)
# (X_test, y_test) = _load_data(testScores[0:], n_prev=50)
# X_test
(X_train, y_train), (X_test, y_test) = train_test_split(allParts, slice_size=40) 

In [113]:
model.fit(X_train, y_train, batch_size=100, epochs = 20, validation_split=0.05)

Train on 680 samples, validate on 36 samples
Epoch 1/20
680/680 [==============================] - 3s 5ms/step - loss: 669.3225 - val_loss: 3902.9792
Epoch 2/20
680/680 [==============================] - 3s 5ms/step - loss: 665.1267 - val_loss: 3812.7136
Epoch 3/20
680/680 [==============================] - 3s 5ms/step - loss: 658.0237 - val_loss: 3728.0496
Epoch 4/20
680/680 [==============================] - 3s 5ms/step - loss: 640.6638 - val_loss: 3797.6067
Epoch 5/20
680/680 [==============================] - 3s 5ms/step - loss: 635.0274 - val_loss: 3550.7742
Epoch 6/20
680/680 [==============================] - 3s 5ms/step - loss: 622.1738 - val_loss: 3788.8057
Epoch 7/20
680/680 [==============================] - 3s 5ms/step - loss: 655.3070 - val_loss: 3381.4912
Epoch 8/20
680/680 [==============================] - 3s 5ms/step - loss: 607.2946 - val_loss: 3304.6492
Epoch 9/20
680/680 [==============================] - 3s 5ms/step - loss: 595.2434 - val_loss: 3226.1428
Epoch 10/2

In [114]:
print(X_train.shape)
print(X_test.shape)

predicted = model.predict(X_test)
print(predicted)

# X_test2 = X_test.copy()
# print(X_test2[0].shape)
# print(predicted[0].shape)
# test = np.append(X_test[0], predicted, axis=0)
# #print(.shape)

# predicted2 = model.predict(X_test2)
# print(predicted2[0])

(716, 40, 5)
(44, 40, 5)
[[  5.97061777e+00   7.84240738e-02   7.71510086e+01   1.17820597e+00
    3.90648926e+02]
 [  5.94075394e+00   7.12016001e-02   7.69311752e+01   1.23018360e+00
    3.88993683e+02]
 [  5.97082996e+00   2.26280257e-01   7.71407776e+01   1.04724395e+00
    3.92065186e+02]
 [  5.94048023e+00   1.96084440e-01   7.77117844e+01   9.67641354e-01
    3.95503510e+02]
 [  5.92242432e+00   1.00073125e-02   7.71490479e+01   1.21791685e+00
    3.90071930e+02]
 [  5.93130350e+00   2.73358021e-02   7.70589218e+01   1.21829426e+00
    3.89542114e+02]
 [  5.93997526e+00   2.67818328e-02   7.68402863e+01   1.22775865e+00
    3.88165436e+02]
 [  5.91962242e+00   2.96624396e-02   7.66190033e+01   1.22463322e+00
    3.86949249e+02]
 [  5.90579653e+00   3.10853999e-02   7.66007156e+01   1.21922469e+00
    3.86817596e+02]
 [  5.97648621e+00   1.26938492e-01   7.61291733e+01   1.19210207e+00
    3.82859100e+02]
 [  6.20990181e+00   3.78611475e-01   7.55323029e+01   9.89012420e-01
    3